In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

## Here are our functions "from the math."

In [ ]:
# we're assuming a form of a/sqrt(x) + b/(x**2) + c
# We're applying the following constraint: a**2+b**2=1: the unit circle. c is free.
# Loss function: mean square error.
# new coordinates: a=cos(alpha), b=sin(alpha), c=beta.

In [ ]:
def MSELoss(p,X,y): # This is the 1-dimensional MSE loss in the original coordinates.
    N = len(y)
    term = 1/N*sum([(y[i]-p[0]/np.sqrt(X[i])-p[1]/(X[i]**2) - p[2])**2 for i in range(N)])
    return term

In [ ]:
def myGrad2(p,X,y): # This assumes that the original loss function is the MSE.
    N = len(y)
    alpha = p[0]
    beta = p[1]
    s1 = sum([ ((1-2*(np.cos(alpha))**2)*X[i]**(3/2) + np.sin(alpha)*(beta-y[i])*X[i]**(7/2) 
                - np.cos(alpha)*((1-X[i]**3)*np.sin(alpha) + X[i]**2*(beta-y[i])))/(X[i]**4)
        for i in range(N)]) # diff wrt alpha.
    s2 = sum([ ((beta-y[i])*X[i]**(5/2) + np.cos(alpha)*X[i]**2 + np.sin(alpha)*np.sqrt(X[i])) / (X[i]**(5/2)) 
              for i in range(N)]) # diff wrt beta
    v1 = -2*s1
    v2 = 2*s2
    ans = np.array([v1/N,v2/N])
    return ans # the components of the gradient of the MSE in our coordinates.

In [ ]:
def myUpdate2(p,v):
    
    def alphaMove(p2,v2,t):
        c3 = (-v2)
        c4 = p2
        return c3*t + c4
    
    def betaMove(p1,v1,t):
        c1 = (-v1)
        c2 = p1
        return c1*t+c2
    
    ans = np.array([alphaMove(p[0],v[0],1.0), betaMove(p[1],v[1],1.0)])
    return ans

In [ ]:
def gradUpdate2(p,X,y,eta):
    
    gradP = myGrad2(p,X,y)
    #print(gradP)
    gradPlr = eta*gradP
    #print(gradPlr)
    newP = myUpdate2(p,gradPlr)
    return newP

## Here is our generic SGD algorithm

In [ ]:
"""
X - X data points
y - y data points
pt - Initial point to start at
Update - Update function to get a new point
epochs - Number of iterations to narrow down the best point
batchsize - The size of the sliced dataset to use in the update function
lr - The learning rate
"""
def SGD(X, y, pt, Update, epochs, batchsize=64, lr=0.01):
	pts = [pt]
	xBatch = X
	yBatch = y
	for _ in range(epochs):
		if len(X) > batchsize:
			# We want to get a random list of integers to get a batch
			xBatch = []
			yBatch = []
			randList = random.sample(range(0, len(X)), batchsize)
			for index in randList:
				xBatch.append(X[index])
				yBatch.append(y[index])
		# Call passed in function TODO: Only will work with gradupdate1, need a way to have it work with multiple.
		pt = Update(pt,xBatch,yBatch,lr)
		for num in pt:
			if math.isnan(num):
				print("Sorry, unable to get an updated point. Please change learning rate and/or starting point.")
				lastPoint = pts[len(pts)-1]
				print(f"Returning the last succesful point: {lastPoint}")
				return lastPoint
		pts.extend([pt]) # Track the path in the original coordinates.
	return pts

## Validate our math formulas on a dummy dataset

In [ ]:
Xtest = np.linspace(1,11,100)
ytest = [1/np.sqrt(x)+1 for x in Xtest] # So the model is y= 1/sqrt(x) + 0/x**2+1.

In [ ]:
pt = [(np.pi)/4,0] # Initialize (new coordinates). This is (exp(2)+1, 0) in the original coordinates.
epochs = 500 # How long to train for?
myeta = 0.1 # Careful with the learning rate: if it's too big, you can run into a "bad point!"
pts = SGD(Xtest, ytest, pt, gradUpdate2, epochs, lr=myeta)
    
opts = np.array([np.array([np.cos(p[0]),np.sin(p[0]),p[1]]) for p in pts])

In [ ]:
print("Starting point in (alpha, beta) coordinates:")
print(pts[0])
print("Ending point in (alpha, beta) coordinates:")
print(pts[-1])
print("starting point in (a,b,c) coordinates:")
print(opts[0])
print("Ending points in (a,b,c) coordinates (goal of (1,0,1)):")
print(opts[-1])

In [ ]:
# Plot the movement of the alpha coordinate along the unit circle (nothing too interesting in beta: goes to 1).
myX = list(np.linspace(0,1,100))
myY = [np.sqrt(1-x**2) for x in myX]
plt.scatter(opts.transpose()[0], opts.transpose()[1], c=[i for i in range(len(opts.transpose()[0]))], cmap='jet')
plt.plot(myX,myY)
plt.show()

In [ ]:
losses = [MSELoss(p,Xtest,ytest) for p in opts]

In [ ]:
# Watch the loss for the first few iterations drop.
plt.plot(losses[0:10])

In [ ]:
# The loss continues to drop.
plt.plot(losses[10:])

In [ ]:
# Bottom line: here is our approximate solution:
print(opts[-1])

## Now fit our curve to real data